In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import matplotlib.pyplot as plt
import re

# URL base de la sección de política de Menéame
BASE_URL = "https://www.meneame.net/m/politica"
PAGES_TO_SCRAPE = 300  # Número de páginas a recorrer

In [5]:


def initialize_driver():
    """
    Inicializa el driver de Selenium con configuración para ejecución en modo headless.
    """
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(options=options)

# Inicializa el driver y espera de Selenium
driver = initialize_driver()
wait = WebDriverWait(driver, 10)

# Inicializar listas para almacenar los datos recopilados
titles, dates, contents, votes, urls = [], [], [], [], []

try:
    for page in range(1, PAGES_TO_SCRAPE + 1):
        # Construye la URL de la página actual
        url = BASE_URL if page == 1 else f"{BASE_URL}?page={page}"
        driver.get(url)
        time.sleep(10)  # Esperar a que la página cargue completamente

        # Procesar el HTML de la página con BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        # Buscar todas las noticias en la página
        news_items = soup.select("div.news-summary")

        skip_next = False  # Bandera para omitir noticias con advertencias
        for item in news_items:
            # Verificar si la noticia tiene una advertencia
            warning = item.find("div", class_="warn tooltip")
            if warning:
                skip_next = True
                continue

            if skip_next:
                skip_next = False
                continue

            # Extraer el título y la URL
            title_tag = item.find("h2").find("a")
            title = title_tag.get_text(strip=True)
            article_url = "https://www.meneame.net" + title_tag["href"]
            titles.append(title)
            urls.append(article_url)

            # Extraer la fecha
            date = item.find("span", class_="ts visible").get_text(strip=True)
            dates.append(date)

            # Extraer el contenido
            content = item.find("div", class_="news-content").get_text(strip=True) if item.find("div", class_="news-content") else ""
            contents.append(content)

            # Extraer el número de meneos
            votes_tag = item.find("a", id=lambda x: x and x.startswith("a-votes"))
            vote_count = int(votes_tag.get_text(strip=True)) if votes_tag else 0
            votes.append(vote_count)

        # Pausa entre cargas de páginas
        time.sleep(5)

finally:
    driver.quit()


In [ ]:

# Crear un DataFrame con los datos recopilados
news_data = pd.DataFrame({
    'Título': titles,
    'Fecha': dates,
    'Contenido': contents,
    'Meneos': votes,
    'URL': urls
})

# Análisis básico del contenido
news_data["Palabras"] = news_data["Contenido"].str.count(" ") + 1  # Contar número de palabras en el contenido

# Visualizar distribución del número de palabras
plt.hist(news_data["Palabras"], bins=100)
plt.xlabel("Número de palabras")
plt.ylabel("Frecuencia")
plt.title("Distribución del número de palabras en las noticias")
plt.show()

# Filtrar noticias con menos de 35 palabras
news_data = news_data[news_data["Palabras"] > 35]

# Guardar los datos en un archivo CSV
news_data.to_csv("noticias_meneame.csv", index=False)
print("Datos guardados en 'noticias_meneame.csv'")

# Cargar y limpiar el archivo CSV
df = pd.read_csv("noticias_meneame.csv")
df['Contenido'] = df['Contenido'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])  # Eliminar enlaces dentro del contenido

# Filtrar noticias que contengan "Viñeta" en el contenido
df = df[df["Contenido"].str.contains("Viñeta") == False]

# Mostrar información básica del DataFrame final
print(df.head())
print(f"Número total de noticias procesadas: {df.shape[0]}")